<a href="https://colab.research.google.com/github/Seungminin/DeepFake_detection_SING/blob/master/DeepFake_detection_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob #큰 데이터 셋을 다룰 때 특정 확장자명을 리스트 형태로 반환해주는 모듈이다.
import zipfile
import cv2

In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/datasets

/content/drive/MyDrive/datasets


In [ ]:
#파이썬 코드를 이용한 zip파일 대용량 파일을 압축해제할 때는 전부 압축해제가 되지 않는다.
#zip_file = zipfile.ZipFile('test.zip') # 압축을 해제할 '/파일경로/파일명.zip'
#zip_file.extractall('/content/drive/MyDrive/datasets/') # 압축을 해제할 '/위치경로/'

In [ ]:
#Linux 코드를 이용한 압축해제, 느리지만 전부 압축해제를 해준다.
!unzip -qq /content/drive/MyDrive/datasets/test.zip -d /content/drive/MyDrive/datasets/test/

In [ ]:
!unzip -qq /content/drive/MyDrive/datasets/train.zip -d /content/drive/MyDrive/datasets/valid/

In [ ]:
test_real_images = list(glob.glob('/content/drive/MyDrive/datasets/test/real/*.jpg'))
test_fake_images = list(glob.glob('/content/drive/MyDrive/datasets/test/fake/*.jpg'))

print(len(test_real_images))
print(len(test_fake_images))

In [ ]:
train_real_images = list(glob.glob('/content/drive/MyDrive/datasets/train/real/*.jpg'))
train_fake_images = list(glob.glob('/content/drive/MyDrive/datasets/train/fake/*.jpg'))

print(len(train_real_images))
print(len(train_fake_images))

In [ ]:
#이미지의 크기를 맞춰주는 작업
def Image_resize(images, target_size):
    resized_images = []
    for image_path in images:
        try:
            image = cv2.imread(image_path)
            resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
            resized_images.append(resized_image)
        except Exception as e:
            print(f"Error resizing image {image_path}: {e}")
    return resized_images

real_images = glob.glob('/content/drive/MyDrive/datasets/test/real/*.jpg')
fake_images = glob.glob('/content/drive/MyDrive/datasets/test/fake/*.jpg')

resized_real_images = Image_resize(real_images, (120, 120))
resized_fake_images = Image_resize(fake_images, (120, 120))


KeyboardInterrupt: 

In [ ]:
print(len(resized_real_images))

3439


In [ ]:
#Cropping은 이미지에서 필요로 하지 않은 Outlier들을 제거하는 방법이다. 전처리의 성능을 높이기 위한 방법!
#이미지를 확대하거나 자르거나 변형시키는 과정.
#https://en.wikipedia.org/wiki/Cropping_(image) 위키참고.

#DeepFake 모델의 성능을 높이기 위해서는 Cropping 기법 중 Center Crop(=얼굴 부분, 중요한 부분)을 주로 사용.

def drop_out_area(size = 300):
  ret = np.ones((8,size,size), int)

  temp = np.ones((size,size), int)

  for i in range(size//2):
    for j in range(i+1):
      temp[i, j] = 0

  for i in range(8):
    ret[i] = temp
    temp = np.rot90(temp)
    if i == 3:
      temp = np.flip(temp, axis = 0)
  return ret